In [1]:
import pandas as pd
import csv

Load the dataset after generated using wikiWho for authorship and segmentation

In [2]:
ds = pd.read_csv('../dataset/wiki/opinions.csv')
ds.shape

(17243, 10)

add language and datetime

In [3]:
ds.loc[ds.page_title.str.startswith('Disc'), 'lang'] = 'es'
ds.loc[ds.page_title.str.startswith('Talk'), 'lang'] = 'en'
ds['creation_dt'] = pd.to_datetime(ds['timestamp'], unit='s')

In [4]:
ds['revision_uid'] = ds.apply(lambda row: '-'.join([str(row['page_id']), str(row['revision_id'])]) , axis=1) 
ds['turn_uid'] = ds.apply(lambda row: '-'.join([str(row['revision_uid']), str(row['turn_id'])]) , axis=1) 

filter opinion with types detected such as: HMTL tags , or bots edits

In [5]:
ds = ds[~ds.type.str.startswith('auto_')]
ds.shape

(12104, 14)

add country info using the page url to match countries dataset

In [6]:
def build_url(row):
    url = row['page_title'].replace(' ', '_')
    url = 'https://en.wikipedia.org/wiki/' + url if row['lang'] == 'en' else 'https://es.wikipedia.org/wiki/' + url
    return url

ds['page_url'] = ds.apply(lambda row: build_url(row), axis=1)
countries = pd.read_csv('../dataset/wiki/countries.csv')
ds2 = pd.merge(ds, countries, left_on='page_url', right_on='wiki', how='left', suffixes=('', '_y'))
del ds2['lang_y']
del ds2['wiki']

In [7]:
ds2.to_csv('../dataset/wiki/opinions_preprocessed.csv', index=False, quoting=csv.QUOTE_ALL)